**Installs and imports**

In [14]:
%pip install ijson
%pip install arabert


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: ijson in c:\users\kewi\appdata\local\programs\python\python310\lib\site-packages (3.2.0.post0)



You should consider upgrading via the 'c:\Users\kewi\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\kewi\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [15]:
import os
import json
import ijson
import pandas as pd
import os
import json
from transformers import AutoTokenizer

**Read the data from** https://github.com/adelmeleka/AQAD

In [3]:
with open('AQAD-master/AQQAD 1.0/FINAL_AAQAD-v1.0.json','rb') as f:
  data = json.load(f)

for allData in data['data']:
    print(allData['title']+"\n")
    print(allData['paragraphs'])
    break

بيونسيه

[{'qas': [{'question': 'متى بدأت بيونسي تصبح شعبية؟', 'id': 1, 'answers': [{'text': 'في أواخر التسعينات', 'answer_start': 220}], 'is_impossible': False}, {'question': 'ما هي المجالات التي تنافس عليها بيونسيه عندما كانت تكبر؟', 'id': 2, 'answers': [{'text': 'غناء ورقص', 'answer_start': 173}], 'is_impossible': False}, {'question': 'متى غادرت بيونسي طفل القدر وتصبح مغنية منفردة؟', 'id': 3, 'answers': [{'text': '(2003)،', 'answer_start': 515}], 'is_impossible': False}, {'question': 'في أي مدينة و ولاية نشأت بيونسيه؟', 'id': 4, 'answers': [{'text': 'هيوستن بولاية تكساس،', 'answer_start': 88}], 'is_impossible': False}, {'question': 'في أي عقد أصبحت بيونسي مشهورة؟', 'id': 5, 'answers': [{'text': 'أواخر التسعينات', 'answer_start': 223}], 'is_impossible': False}, {'question': 'في أي مجموعة R&B كانت هي المغنية الرئيسية؟', 'id': 6, 'answers': [{'text': 'دستنيز تشايلد', 'answer_start': 290}], 'is_impossible': False}, {'question': 'ما الألبوم الذي جعلها فنانة معروفة عالميا؟', 'id': 7, 'ans

In [4]:
def parse_data(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    # initialize lists for contexts, questions, and answers
    contexts = []
    questions = []
    answers = []
    # iterate through all data in squad data
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if 'plausible_answers' in qa.keys():
                    access = 'plausible_answers'
                else:
                    access = 'answers'
                for answer in qa['answers']:
                    # append data to lists
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    # return formatted data lists
    return contexts, questions, answers

In [5]:
contexts, questions, answers = parse_data('AQAD-master/AQQAD 1.0/FINAL_AAQAD-v1.0.json')
#splitting the data into train and test
from sklearn.model_selection import train_test_split
train_contexts, val_contexts, train_questions, val_questions, train_answers, val_answers = train_test_split(contexts, questions, answers, test_size=0.1, random_state=42)

In [6]:
print(len(train_questions))
display(train_questions[0:3])
print(len(train_answers))
display(train_answers[0:3])
print(len(train_contexts))
display(train_contexts[0:3])
print('-----------------------------------------')
print(len(val_questions))
display(val_questions[0:3])
print(len(val_answers))
display(val_answers[0:3])
print(len(val_contexts))
display(val_contexts[0:3])

10343


['ما الذي تم بناء استاد فرنسا من أجله؟',
 'كيف ألقاب الجماعة درع لديه أرسنال؟',
 'في أي اتجاه يُنشر الخشب في كثير من الأحيان بحيث تظهر عقدة كدائرة صلبة تتدفق الحبوب حولها؟']

10343


[{'text': 'كأس العالم لكرة القدم', 'answer_start': 187},
 {'text': '2014،', 'answer_start': 919},
 {'text': 'الألواح بطول 366 سم وبعرض', 'answer_start': 9}]

10343


...

['ما هي البكتيريا التي تسبب عدوى السل؟',
 'ما هي البكتيريا التي تسبب مرض الحمى التي رصدت في جبال روكي؟',
 'كم من السكان الأصليين بالقرب من ولاية ماساتشوستس لقوا حتفهم بسبب الجدري في الوباء بين عامي 1617 و 1619؟']

1150


[{'text': 'السل', 'answer_start': 89},
 {'text': 'أنواع', 'answer_start': 706},
 {'text': '30% من الأمريكيين الأصليين القاطنين في', 'answer_start': 512}]

1150


['من المُقدّر أن حوالي 5 - 10٪ من غير المُصابين بفيروس نقص المناعة البشرية والمصابين بعدوى السل سيصبح مرضهم نشطًا خلال حياتهم وفي المقابل، فإن 30٪ من المُصابين بفيروس نقص المناعة البشرية سيصبح مرضهم نشطًا. قد يُصيب السل أي جزء من الجسم، ولكن إصابته للرئتين تُعتبر الأكثر شيوعًا (وهو ما يُعرف باسم السل الرئوي). يحدث السل خارج الرئوي عندما يتطور مرض السل خارج الرئتين. قد يتشارك السل خارج الرئوي مع السل الرئوي أيضًا  تشمل العلامات والأعراض العامة الحمى والقشعريرة والتعرق الليلي وفقدان الشهية وفقدان الوزن والتعب،  وقد يحدث تعجّر الأصابع الواضح أيضًا.\n',
 'يمتلك كل نوع من الأمراض مميزات خاصة به تمكنه من التفاعل مع مستقبلاته في الجسم البشري. فبعض الكائنات الحية مثل المكورات العقدية يمكن أن تسبب الالتهابات الجلدية، والالتهاب الرئوي، والسحايا، وتسمم الدم حيث ينتج صدمة فتتوسع وتتضخم الأوعية مما يسبب الموت. مع ذلك فإن هذه الكائنات الحية هي أيضا جزء من الإنسان، وعادةً ما تتواجد على الجلد أو في الأنف من دون أن تسبب أي مرض على الإطلاق. وهناك كائنات أخرى دائماً تسبب الأمراض لدى البشر مثل الريكتسية و

**Importing Arabert processor from** https://github.com/aub-mind/arabert

**Preprocessing the dataset using arabert in addition to this SQuAD preprocessing guide** https://www.youtube.com/watch?v=ZIRmXkHp0-c&t=287s

In [7]:
def add_end_idx(answers, contexts):
    # loop through each answer-context pair
    for answer, context in zip(answers, contexts):
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
        else:
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    # this means the answer is off by 'n' tokens
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [8]:
#loop over train answers and train contexts and make sure that they match
def check_answers_contexts_match(answers, contexts, questions, show_anamolies=False):
    
    answers_without_answer_end_key =0
    non_matching_answers=0
    for answer, context, question in zip(answers, contexts, questions):
        
        #check if context has key answer_end and if not 
        if 'answer_end' not in answer.keys():
            answers_without_answer_end_key +=1
            if show_anamolies:
                print(question)
                print(answer)
                print(context[answer['answer_start']:answer['answer_start']+ len(answer['text'])+1])
            continue 

        #check if answer matches context
        if context[answer['answer_start']:answer['answer_end']] != answer['text']:
            non_matching_answers +=1
    display('answers without answer_end key:', answers_without_answer_end_key,'non matching answers:', non_matching_answers)
    
check_answers_contexts_match(train_answers, train_contexts, train_questions)
check_answers_contexts_match(val_answers, val_contexts, val_questions)

'answers without answer_end key:'

35

'non matching answers:'

0

'answers without answer_end key:'

6

'non matching answers:'

0

In [16]:
arabert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabert",
                                                    do_lower_case=False,
                                                    do_basic_tokenize=True,
                                                    remove_html_markup = False,
                                                    strip_tashkeel = True,
                                                    strip_tatweel = True,
                                                    keep_emojis = True,
                                                    insert_white_spaces = True,
                                                    replace_slash_with_dash = True,
                                                    apply_farasa_segmentation = True)

**Encoding**

In [32]:
def show_encoding_of(sentence, tokenizer):
    encoding = tokenizer(sentence)
    
    print('length of encoding:', len(encoding['input_ids']))
    print(' number of words around splitted spaces :', len(sentence.split(' ')))
    list_of_tokens = []
    count_of_unk_tokens = 0
    for i in encoding['input_ids']:
        #print the decoding of it
        list_of_tokens.append(tokenizer.decode([i]))
        if(tokenizer.decode([i]) == '[UNK]'):
            count_of_unk_tokens +=1
    #print the list of tokens
    print('number of tokens that are not in the vocab:', count_of_unk_tokens)
    print(sentence)
    # loop over the list of tokens and print them
    # for token in list_of_tokens:
    #     display(token)
    print(list_of_tokens[1:-1])
    # print(encoding)
    
# test_text = "الألواح الخشبية تعالج بعد ذلك في أفران للتحميص حتى لا تتأثر مستقبلاً بتغييرات درجات الحرارة أو الرطوبة الموجودة في الجو"
# test_text = "يوجد في باريس العديد من الملاعب المخصصة لمختلف أنواع الرياضات. يعدّ ملعب فرنسا الذي يتسع لأكثر من 80 ألف متفرج أكبر ملاعب البلاد، وكان قد بني هذا الملعب الواقع في منطقة سان دينس لاستضافة كأس العالم لكرة القدم 1998 والذي حازت عليه فرنسا للمرة الأولى في تاريخها. يستخدم الملعب لممارسة كرة القدم، والرجبي، وألعاب القوى. يستضيف الملعب مباريات منتخب فرنسا الوطني للرجبي سنوياً في بطولة الأمم الستة. وكذلك يستضيف مباريات منتخب فرنسا لكرة القدم الودية وتصفيات البطولات الكبرى. بالإضافة إلى نادي باريس سان جيرمان، تملك المدينة نوادي كرة قدم أخرى مثل: نادي باريس، والنجم الأحمر ونادي فرنسا"
for i in range(5,10):
    test_text = train_contexts[i]
    show_encoding_of(test_text, arabert_tokenizer)


length of encoding: 93
 number of words around splitted spaces : 41
number of tokens that are not in the vocab: 0
استضافت المطربة المصرية أم كلثوم احتفال الجمهور بعودة الضباط رغم تحفظات الحكومة الملكية، التي كانت قد تعرضت لضغوط من قبل الحكومة البريطانية لمنع الاستقبال، وزاد ذلك من عزم عبد الناصر على الإطاحة بالملكية. بدأ عبد الناصر كتابة "فلسفة الثورة" أثناء الحصار.

['استضاف', '##ت', 'ال', '##مطر', '##ب', '##ة', 'ال', '##مصري', '##ة', 'أم', 'كلثوم', 'احتفال', 'ال', '##جمهور', 'ب', '##عود', '##ة', 'ال', '##ضب', '##اط', 'رغم', 'تحفظ', '##ات', 'ال', '##حكوم', '##ة', 'ال', '##ملكي', '##ة', '،', 'التي', 'كان', '##ت', 'قد', 'تعرض', '##ت', 'ل', '##ض', '##غوط', 'من', 'قبل', 'ال', '##حكوم', '##ة', 'ال', '##بريطاني', '##ة', 'لم', '##ن', '##ع', 'ال', '##اس', '##ت', '##قب', '##ال', '،', 'و', '##زاد', 'ذلك', 'من', 'عزم', 'عبد', 'ال', '##ناصر', 'على', 'ال', '##إ', '##طاح', '##ة', 'بالم', '##لك', '##ي', '##ة', '.', 'بدأ', 'عبد', 'ال', '##ناصر', 'كتاب', '##ة', '"', 'فلسف', '##ة', 'ال', '##ثور', '##ة'

In [11]:
tokenizer = arabert_tokenizer
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [12]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        
        # old
        # start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        # end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))
        # Mine mah, lol I am sure it must work
        start_positions.append(encodings.char_to_token(i, answers[i].get('answer_start', None)))
        end_positions.append(encodings.char_to_token(i, answers[i].get('answer_end', None)))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift one token forward
        go_back = 1
        print(end_positions[-1])
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-go_back)
            go_back +=1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# apply function to our data
add_token_positions(train_encodings, train_answers)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2
None
None
None
None
None
None
None
None
None
150
None
None
None
123
None
None
None
198
None
None
None
20
None
None
None
None
None
None
None
None
None
144
None
None
None
None
None
136
None
None
None
None
40
None
None
None
None
None
148
None
None
None
26
None
None
None
213
None
None
None
201
None
None
None
158
140
133
61
None
None
None
None
None
None
None
None
None
None
None
None
54
None
None
None
None
None
80
None
83
None
None
None
None
None
None
None
None
None
58
None
None
None
None
None
None
None
105
None
None
37
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
18
None
160
None
None
None
None
None
None
None
None
None
None
19
None
None
None
None
None
None
108
None
None
None
None
None
None
None
None
None
None
68
None
None
None
None
121
None
None
None
None
None
None
None
None
None
None
None
None
No

KeyError: 'answer_end'

In [ ]:
train_encodings['start_positions'][0:20]

**Importing the model**

**Finetuning**

**Evaluation**

**Saving the model**